In [2]:
import pandas as pd

In [3]:
# # Можно было сделать так, но запоминать индексы сложнее, чем названия 

# import os
# dfs = []
# for file_nm in os.listdir('datasets/'):
#     if 'D_' in file_nm:
#         df = pd.read_csv('datasets/' + file_nm)
#         dfs.append(df)


In [4]:
clients = pd.read_csv('datasets/D_clients.csv')
close_loan = pd.read_csv('datasets/D_close_loan.csv')
job = pd.read_csv('datasets/D_job.csv')
last_credit = pd.read_csv('datasets/D_last_credit.csv')
loan = pd.read_csv('datasets/D_loan.csv')
pens = pd.read_csv('datasets/D_pens.csv')
salary = pd.read_csv('datasets/D_salary.csv')
target = pd.read_csv('datasets/D_target.csv')
work = pd.read_csv('datasets/D_work.csv')

In [5]:
# Убираем все дубликаты из таблиц

dfs = [clients, close_loan, job, last_credit, loan, pens, salary, target, work]
for df in dfs:
    df.drop_duplicates(inplace=True)

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


In [6]:
# Отдельно рассчитываю агрегаты ссуд на клиента
loans_table = pd.merge(loan, close_loan, on='ID_LOAN')
loans_table = loans_table.groupby('ID_CLIENT').agg({'CLOSED_FL':['count','sum']})
loans_table.columns = ['LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED']

In [7]:
# Собираю базовую таблицу

t1 = pd.merge(target, 
              clients[['ID', 'AGE', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL','GENDER','CHILD_TOTAL','DEPENDANTS']],
              left_on='ID_CLIENT', 
              right_on='ID',
              how='left'
              ).drop(['ID'], axis=1)

t2 = pd.merge(t1,
              salary[['ID_CLIENT', 'PERSONAL_INCOME']],
              on='ID_CLIENT',
              how='left')

t3 = pd.merge(t2, loans_table, on='ID_CLIENT')
t3.to_csv('datasets/D_basic.csv')

In [8]:
t3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15223 entries, 0 to 15222
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AGREEMENT_RK       15223 non-null  int64  
 1   ID_CLIENT          15223 non-null  int64  
 2   TARGET             15223 non-null  int64  
 3   AGE                15223 non-null  int64  
 4   SOCSTATUS_WORK_FL  15223 non-null  int64  
 5   SOCSTATUS_PENS_FL  15223 non-null  int64  
 6   GENDER             15223 non-null  int64  
 7   CHILD_TOTAL        15223 non-null  int64  
 8   DEPENDANTS         15223 non-null  int64  
 9   PERSONAL_INCOME    15223 non-null  float64
 10  LOAN_NUM_TOTAL     15223 non-null  int64  
 11  LOAN_NUM_CLOSED    15223 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 1.5 MB
